In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [ ]:
def aic_score(model, X, y):
    """Calcula el AIC dado un modelo ya ajustado."""
    y_pred = model.predict(X)
    n = len(y)
    resid = y - y_pred
    rss = np.sum(resid ** 2)
    k = len(model.coef_) + 1
    aic = n * np.log(rss / n) + 2 * k
    return -aic

def bic_score(model, X, y):
    """Calcula el BIC dado un modelo ya ajustado."""
    y_pred = model.predict(X)
    n = len(y)
    resid = y - y_pred
    rss = np.sum(resid ** 2)
    k = len(model.coef_) + 1
    bic = n * np.log(rss / n) + k * np.log(n)
    return -bic

def r2_adj_score(model, X, y):
    """Calcula el R² ajustado dado un modelo ya ajustado."""
    n = len(y)
    p = len(model.coef_)
    r2 = model.score(X, y)
    r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return r2_adj

def score_function_wrapper(metric):
    metrics = {'aic': aic_score, 'bic': bic_score, 'r2_adj': r2_adj_score}
    if metric not in metrics:
        raise ValueError(f"{metric} is not a valid metric. Choose from {list(metrics.keys())}.")
    return metrics[metric]



def eval_model(df, vars):
    X = df[vars]
    y = df['y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2025)
    lr = LinearRegression()
    model = lr.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    MSE = (np.mean(np.square(y_test - y_preds)))
    print(f'The model MSE is: {MSE:.3f}')


In [ ]:
X, y, coef = make_regression(
    n_samples = 1200,
    n_features = 15,
    n_informative = 8,
    noise = 4.5,
    coef=True,
    random_state = 2025
)

columns = [f'X{i + 1}' for i in range(X.shape[1])]
df = pd.DataFrame(X, columns = columns)
df['y'] = y


In [ ]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,y
0,-0.782887,-0.504763,-0.513559,0.758179,0.316008,-1.465977,0.372124,-0.037389,0.561418,1.509250,-0.703625,0.437332,0.869750,-0.456573,0.964083,40.493499
1,-0.153812,1.945519,-1.260359,-0.102555,-0.821676,-1.061396,0.541510,-0.614084,-0.210228,-0.096182,-0.731476,-0.902293,0.480737,-0.328865,-0.401505,-168.016037
2,-0.736576,-0.634103,-1.655245,0.988401,0.113244,0.142174,0.435258,-0.384776,-1.299637,1.452306,-1.311151,0.640503,-0.415986,-0.336106,-2.136941,-101.976337
3,-0.345572,-1.761674,0.338704,0.420485,-0.265529,-0.487165,0.664500,0.315319,0.961425,-1.351213,-0.408434,0.894105,-0.295988,-0.302159,0.616066,12.278358
4,1.418499,-1.765933,-0.085609,-0.487351,0.776347,-1.465167,0.028054,-0.585820,1.552118,-0.018196,-1.075876,-0.167844,0.278455,-0.701796,0.707887,-167.871647


In [ ]:
X = df.drop(['y'], axis=1)
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2025)


lr = LinearRegression()


sfs1 = SequentialFeatureSelector(
    estimator=lr,
    k_features='best',
    forward=False,
    floating=False,
    scoring=score_function_wrapper('aic'),
    cv=10,
    n_jobs=-1
)

sfs1.fit(X_train, y_train)
# Extraer resultados
pd.DataFrame.from_dict(sfs1.get_metric_dict()).T



,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
15,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-284.3380452907436, -319.9250544752684, -287....",-302.692929,"(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11,...",10.579077,14.243829,4.747943
14,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14)","[-282.4330733690931, -316.8925857826372, -285....",-300.401893,"(X1, X2, X3, X4, X5, X6, X8, X9, X10, X11, X12...",10.413678,14.021135,4.673712
13,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14)","[-280.8746561060791, -315.1727917505726, -283....",-298.131897,"(X1, X2, X3, X4, X5, X6, X8, X9, X10, X11, X12...",10.471892,14.099515,4.699838
12,"(1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14)","[-277.35152646237145, -313.2875886959984, -280...",-295.933298,"(X2, X3, X4, X5, X6, X8, X9, X10, X11, X12, X1...",10.654298,14.345109,4.781703
11,"(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14)","[-275.14791775377387, -311.15349648758723, -27...",-293.737748,"(X2, X3, X4, X6, X8, X9, X10, X11, X12, X13, X15)",10.640911,14.327085,4.775695
10,"(1, 2, 3, 5, 7, 9, 10, 11, 12, 14)","[-273.5352413685891, -308.5875031227415, -277....",-291.606287,"(X2, X3, X4, X6, X8, X10, X11, X12, X13, X15)",10.466693,14.092514,4.697505
9,"(1, 2, 3, 5, 7, 9, 10, 11, 14)","[-271.5072175823201, -306.95489958509916, -275...",-289.521168,"(X2, X3, X4, X6, X8, X10, X11, X12, X15)",10.278026,13.838491,4.61283
8,"(1, 2, 3, 5, 7, 9, 11, 14)","[-270.39188530954095, -305.2019902654966, -274...",-287.700329,"(X2, X3, X4, X6, X8, X10, X12, X15)",9.922616,13.359961,4.45332
7,"(1, 2, 3, 5, 9, 11, 14)","[-460.84722181983466, -489.8684999408145, -495...",-488.779342,"(X2, X3, X4, X6, X10, X12, X15)",9.967048,13.419785,4.473262
6,"(1, 2, 3, 5, 9, 11)","[-680.9292713179474, -661.2105344080103, -677....",-667.867533,"(X2, X3, X4, X6, X10, X12)",7.766327,10.4567,3.485567


In [ ]:
vars_selected = list(sfs1.k_feature_names_)
eval_model(df, vars_selected)
print(f'the model vars selected are: {vars_selected}, number is: {len(vars_selected)}')




The model MSE is: 21.793
the model vars selected are: ['X2', 'X3', 'X4', 'X6', 'X8', 'X10', 'X12', 'X15'], number is: 8


In [ ]:
lr = LinearRegression()


sfs2 = SequentialFeatureSelector(
    estimator=lr,
    k_features='best',
    forward=False,
    floating=False,
    scoring=score_function_wrapper('bic'),
    cv=10,
    n_jobs=-2
)

sfs2.fit(X_train, y_train)

# Extraer resultados
pd.DataFrame.from_dict(sfs2.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
15,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-324.33500001602783, -359.9220092005526, -327...",-342.689884,"(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11,...",10.579077,14.243829,4.747943
14,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14)","[-319.9302184240471, -354.3897308375912, -322....",-337.899038,"(X1, X2, X3, X4, X5, X6, X8, X9, X10, X11, X12...",10.413678,14.021135,4.673712
13,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14)","[-315.8719914907028, -350.17012713519637, -318...",-333.129233,"(X1, X2, X3, X4, X5, X6, X8, X9, X10, X11, X12...",10.471892,14.099515,4.699838
12,"(1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14)","[-309.8490521766649, -345.78511441029184, -313...",-328.430823,"(X2, X3, X4, X5, X6, X8, X9, X10, X11, X12, X1...",10.654298,14.345109,4.781703
11,"(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14)","[-305.1456337977371, -341.1512125315504, -308....",-323.735464,"(X2, X3, X4, X6, X8, X9, X10, X11, X12, X13, X15)",10.640911,14.327085,4.775695
10,"(1, 2, 3, 5, 7, 9, 10, 11, 12, 14)","[-301.033147742222, -336.08540949637444, -304....",-319.104194,"(X2, X3, X4, X6, X8, X10, X11, X12, X13, X15)",10.466693,14.092514,4.697505
9,"(1, 2, 3, 5, 7, 9, 10, 11, 14)","[-296.5053142856227, -331.9529962884018, -300....",-314.519265,"(X2, X3, X4, X6, X8, X10, X11, X12, X15)",10.278026,13.838491,4.61283
8,"(1, 2, 3, 5, 7, 9, 11, 14)","[-292.8901723425134, -327.70027729846896, -297...",-310.198616,"(X2, X3, X4, X6, X8, X10, X12, X15)",9.922616,13.359961,4.45332
7,"(1, 2, 3, 5, 9, 11, 14)","[-480.8456991824768, -509.8669773034566, -515....",-508.777819,"(X2, X3, X4, X6, X10, X12, X15)",9.967048,13.419785,4.473262
6,"(1, 2, 3, 5, 9, 11)","[-698.4279390102593, -678.7092021003222, -694....",-685.366201,"(X2, X3, X4, X6, X10, X12)",7.766327,10.4567,3.485567


In [ ]:
vars_selected = list(sfs2.k_feature_names_)
eval_model(df, vars_selected)
print(f'the model vars selected are: {vars_selected}, number is: {len(vars_selected)}')

The model MSE is: 21.793
the model vars selected are: ['X2', 'X3', 'X4', 'X6', 'X8', 'X10', 'X12', 'X15'], number is: 8


In [ ]:
lr = LinearRegression()


sfs3 = SequentialFeatureSelector(
    estimator=lr,
    k_features='best',
    forward=False,
    floating=False,
    scoring=score_function_wrapper('r2_adj'),
    cv=10,
    n_jobs=-3
)

sfs3.fit(X_train, y_train)

# Extraer resultados
pd.DataFrame.from_dict(sfs3.get_metric_dict()).T


,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
15,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.9990989818196526, 0.999067418043303, 0.9992...",0.999035,"(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11,...",0.000158,0.000213,0.000071
14,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14)","[0.9991100562285575, 0.9990903479696246, 0.999...",0.999052,"(X1, X2, X3, X4, X5, X6, X8, X9, X10, X11, X12...",0.000155,0.000209,0.00007
13,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14)","[0.9991174463936683, 0.9990995178743349, 0.999...",0.999066,"(X1, X2, X3, X4, X5, X6, X8, X9, X10, X11, X12...",0.000154,0.000208,0.000069
12,"(0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14)","[0.9991306588031779, 0.9991124507444027, 0.999...",0.999081,"(X1, X2, X3, X4, X6, X8, X9, X10, X11, X12, X1...",0.000152,0.000204,0.000068
11,"(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14)","[0.9991564171974703, 0.9991227952452856, 0.999...",0.999094,"(X2, X3, X4, X6, X8, X9, X10, X11, X12, X13, X15)",0.00015,0.000203,0.000068
10,"(1, 2, 3, 5, 7, 9, 10, 11, 12, 14)","[0.9991635032522207, 0.999139328760445, 0.9993...",0.999107,"(X2, X3, X4, X6, X8, X10, X11, X12, X13, X15)",0.000147,0.000199,0.000066
9,"(1, 2, 3, 5, 7, 9, 10, 11, 14)","[0.9991742166302334, 0.9991466105600155, 0.999...",0.99912,"(X2, X3, X4, X6, X8, X10, X11, X12, X15)",0.000143,0.000193,0.000064
8,"(1, 2, 3, 5, 7, 9, 11, 14)","[0.9991763550127675, 0.9991548290381307, 0.999...",0.99913,"(X2, X3, X4, X6, X8, X10, X12, X15)",0.000139,0.000187,0.000062
7,"(1, 2, 3, 5, 9, 11, 14)","[0.9930959592740766, 0.9933568524035188, 0.992...",0.991806,"(X2, X3, X4, X6, X10, X12, X15)",0.001209,0.001627,0.000542
6,"(1, 2, 3, 5, 9, 11)","[0.9195557965002381, 0.9549628758197911, 0.940...",0.939721,"(X2, X3, X4, X6, X10, X12)",0.007844,0.010562,0.003521


In [ ]:
vars_selected = list(sfs3.k_feature_names_)
eval_model(df, vars_selected)
print(f'the model vars selected are: {vars_selected}, number is: {len(vars_selected)}')
print(f'Yhe model is the same as the other two, because this subset was the same')

The model MSE is: 21.793
the model vars selected are: ['X2', 'X3', 'X4', 'X6', 'X8', 'X10', 'X12', 'X15'], number is: 8
Yhe model is the same as the other two, because this subset was the same


# This is what would happend to MSE if we picked a different subset

In [ ]:
vars_selected = ['X2', 'X3', 'X4', 'X6', 'X10', 'X12', 'X15']
eval_model(df, vars_selected)
print(f'the model vars selected are: {vars_selected}, number is: {len(vars_selected)}')

The model MSE is: 210.931
the model vars selected are: ['X2', 'X3', 'X4', 'X6', 'X10', 'X12', 'X15'], number is: 7


# The perfect values for the data were:

In [ ]:
for i, beta_i in enumerate(coef):
    print(f'X{i+1} == {beta_i}')
print(f'we selected the following: {vars_selected}')

X1 == 0.0
X2 == 49.77614479946497
X3 == 89.06268514362664
X4 == 51.905892023881165
X5 == 0.0
X6 == 38.98847527736119
X7 == 0.0
X8 == 13.520468663165897
X9 == 0.0
X10 == 35.70063805379543
X11 == 0.0
X12 == 92.78789728648805
X13 == 0.0
X14 == 0.0
X15 == 37.61234559602978
we selected the following: ['X2', 'X3', 'X4', 'X6', 'X10', 'X12', 'X15']


as we can see the model was able to select only the params that were significative